In [1]:
import cv2

In [2]:
import boto3

In [3]:
import time

In [4]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [5]:
from cvzone.HandTrackingModule import HandDetector

In [6]:
ec2 = boto3.resource('ec2')
elb = boto3.client('elbv2')
allOS=[]

In [7]:
def LaunchOS():
    instances=ec2.create_instances(
        ImageId="ami-0da59f1af71ea4ad2",
        MinCount=1,
        MaxCount=1,
        InstanceType="t2.micro",
        SecurityGroupIds= ['sg-03e07fad3c92c8384',],
    )
    
    OSid = instances[0].id
    allOS.append(OSid)
    
    time.sleep(30)
    
    # print(allOS)
    print("Total OS : " , len(allOS))

In [8]:
def TerminateOS():
    if allOS:
        myos = allOS.pop()
        response = ec2.instances.filter(InstanceIds=[myos]).terminate()
        

        
        print("Remaining OS : " , len(allOS))
        return response
    else:
        print("no more OS is Running")

In [9]:
detector = HandDetector(maxHands=1,
                        detectionCon=0.8)

In [10]:
cap = cv2.VideoCapture(0)

In [ ]:
while True:
    ret,img = cap.read()
    cv2.imshow("Img" , img)
    if cv2.waitKey(1000) == 13:
        break
        
    hand = detector.findHands(img , draw=False)
    if hand:
        lmlist = hand[0]
        if lmlist:
            fingerup = detector.fingersUp(lmlist)
            print(fingerup)
            if fingerup == [0, 1, 0, 0, 0]:
                print("sec finger ..")
                TerminateOS()
            
            elif fingerup == [0, 1, 1, 0, 0]:
                print("2 and 3 finger ..")
                LaunchOS()

cv2.destroyAllWindows

In [ ]:
cap.release